In [ ]:
from pathlib import Path
from dotenv import load_dotenv
from sqlalchemy import create_engine
import os
import pandas as pd
load_dotenv()

### Establishing Connection

In [ ]:
host = os.getenv('DB_HOST')
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
port = os.getenv('DB_PORT')
dbname = os.getenv('DB_NAME')

ENGINE = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}")
ENGINE

In [ ]:
def deleteRows(tableName):
    with ENGINE.connect() as connection:
        connection.execute(f"DELETE FROM {tableName}")
    
def showTable(tableName):
    command = f"SELECT * FROM {tableName}"
    return pd.read_sql(command, ENGINE)

def insertData(tableName, data):
    with ENGINE.connect() as connection:
        data.to_sql(tableName, con=connection, if_exists='append', index=False)

### Veryfying the connection

In [ ]:
# show all the tables in the database
# restart the kernel after changing the .env file
pd.read_sql("SHOW TABLES", ENGINE)

### Adding all tables

In [ ]:
dataFolder = Path.cwd()/'pokedex'/'pokedex'/'data'/'csv'

In [ ]:
command = "ALTER TABLE pokemon_species DROP CONSTRAINT pokemon_species_ibfk_2"
with ENGINE.connect() as connection:
    connection.execute(command)

deleteRows("pokemon_types")
deleteRows("pokemon_moves")
deleteRows("pokemon")
deleteRows("pokemon_species")
deleteRows("pokemon_shapes")
deleteRows("type_efficacy")
deleteRows("pokemon_habitats")
deleteRows("pokemon_colors")
deleteRows("moves")
deleteRows("types")
deleteRows("move_damage_classes")
deleteRows("generations")

command = "ALTER TABLE pokemon_species ADD CONSTRAINT pokemon_species_ibfk_2 FOREIGN KEY (evolves_from_species_id) REFERENCES pokemon_species(id)"
with ENGINE.connect() as connection:
    connection.execute(command)

### 1. generations

In [ ]:
filename = 'generations.csv'
drop_cols = ['id']
rename_cols = {'main_region_id':'id'}

df = pd.read_csv(dataFolder/filename)
df = df.drop(drop_cols, axis=1)
df = df.rename(columns=rename_cols)

df

In [ ]:
insertData('generations', df)

### 2. move_damage_classes

In [ ]:
filename = "move_damage_classes.csv"

df = pd.read_csv(dataFolder/filename)
df

In [ ]:
insertData('move_damage_classes', df)

### 3. types

In [ ]:
filename = "types.csv"
drop_cols = ["generation_id", "damage_class_id"]

df = pd.read_csv(dataFolder/filename)
df = df.drop(drop_cols, axis=1)
df

In [ ]:
insertData('types', df)

### 4. moves

In [ ]:
filename = "moves.csv"
keepcols = [
    "id",
    "identifier",
    "generation_id",
    "type_id",
    "power",
    "pp",
    "accuracy",
    "priority",
    "damage_class_id",
]

df = pd.read_csv(dataFolder/filename)
df = df[keepcols]

df

In [ ]:
insertData('moves', df)

### 5. pokemon_colors

In [ ]:
filename = "pokemon_colors.csv"

df = pd.read_csv(dataFolder/filename)
df

In [ ]:
insertData('pokemon_colors', df)

### 6. pokemon_habitats

In [ ]:
filename = "pokemon_habitats.csv"

df = pd.read_csv(dataFolder/filename)
df

In [ ]:
insertData('pokemon_habitats', df)

### 7. type_efficacy

In [ ]:
filename = "type_efficacy.csv"

df = pd.read_csv(dataFolder/filename)
df

In [ ]:
insertData('type_efficacy', df)

### 8. pokemon_shapes

In [ ]:
filename = "pokemon_shapes.csv"

df = pd.read_csv(dataFolder/filename)
df

In [ ]:
insertData('pokemon_shapes', df)

### 9. pokemon_species

In [ ]:
filename = "pokemon_species.csv"
keepcols = [
    "id",
    "identifier",
    "generation_id",
    "evolves_from_species_id",
    "color_id",
    "shape_id",
    "habitat_id",
    "capture_rate",
    "is_legendary",
    "is_mythical",
]


df = pd.read_csv(dataFolder/filename)
df = df[keepcols]

# check the types of the columns
df.dtypes
castCols = ['evolves_from_species_id', 'shape_id', 'habitat_id']
for col in castCols:
    df[col] = df[col].astype('Int64')

df

In [ ]:
while True:
    command = "select count(*) FROM pokemon_species"
    cnt = pd.read_sql(command, ENGINE).values[0][0]
    expectedRow = df.shape[0]
    if cnt == expectedRow: break
    
    for index, row in df.iterrows():
        with ENGINE.connect() as connection:
            poke_id = row['id']
            poke_identifier = row['identifier']
            generation_id = row['generation_id']
            evolves_from_species_id = row['evolves_from_species_id']
            color_id = row['color_id']
            shape_id = row['shape_id']
            habitat_id = row['habitat_id']
            capture_rate = row['capture_rate']
            is_legendary = row['is_legendary']
            problemcols = ["evolves_from_species_id", "shape_id", "habitat_id"]
            for col in problemcols:
                if pd.isnull(row[col]):
                    row[col] = 'NULL'
            command = f"INSERT INTO pokemon_species VALUES ({row['id']}, '{row['identifier']}', {row['generation_id']}, {row['evolves_from_species_id']}, {row['color_id']}, {row['shape_id']}, {row['habitat_id']}, {row['capture_rate']}, {row['is_legendary']}, {row['is_mythical']})"
            try:
                connection.execute(command)
            except Exception as e:
                pass

### 10. pokemon

In [ ]:
filename = "pokemon.csv"
keepCols = [
    "id",
    "identifier",
    "species_id",
    "height",
    "weight",
    "base_experience",
]
df = pd.read_csv(dataFolder/filename)
df = df[keepCols]
df


In [ ]:
insertData('pokemon', df)

### 11. pokemon_moves

In [ ]:
filename = "pokemon_moves.csv"
keepCols = [
    "pokemon_id",
    "move_id",
    "level",
]
df = pd.read_csv(dataFolder/filename)
df = df[keepCols]
df = df.drop_duplicates()
df

In [ ]:
insertData('pokemon_moves', df)

### 12. pokemon_types

In [ ]:
filename = "pokemon_types.csv"
keepCols = [
    "pokemon_id",
    "type_id",
    "slot",
]

df = pd.read_csv(dataFolder/filename)
df = df[keepCols]
df

In [ ]:
insertData('pokemon_types', df)